In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def f(x):
    return np.tanh(x)
def d_f(x):
    return 1/np.cosh(10*np.tanh(x/10))**2  # the tanh prevents oveflow

class RNN:
    def __init__(self, nIN, nHS, nOUT,ηS=[0.1,0.1,0.1], τ=10):
        np.random.seed(42)
        self.nIN = nIN
        self.nHS = nHS
        self.nOUT = nOUT
        self.ΔOUT = np.zeros((nOUT, nHS))
        self.ΔHS = np.zeros((nHS, nHS))
        self.ΔIN = np.zeros((nHS, nIN))
        self.xI = np.zeros(nIN)
        self.ηS = ηS
        self.τ = τ
        self.h0 = 0
        self.hI = np.zeros(nHS)
        self.hF = np.zeros(nHS)
        self.ht = np.zeros(nHS)
        self.k = 0
        self.u = None

        # Initialize weights:
        self.wIN = 0.1*(np.random.rand(nHS, nIN) - 1)
        self.wHS = 1.5*np.random.randn(nHS, nHS)/nHS**0.5
        self.wOUT = 0.1*(2*np.random.rand(nOUT, nHS) - 1)/nHS**0.5
        self.b = np.random.randn(nHS, nOUT)/nOUT**0.5
        self.p = np.zeros((self.nHS, self.nHS))
        self.q = np.zeros((self.nHS, self.nIN))

        self.wIN = self.xavier_uniform([nHS, nIN])
        self.wHS = self.xavier_uniform([nHS, nHS])
        self.wOUT = self.xavier_uniform([nOUT, nHS])
        self.b = np.random.randn(nHS, nOUT)/nOUT**0.5

    def fit(self, x, y):
        η1,η2,η3 = self.ηS

        self.u = np.dot(self.nHS,self.hI) + np.dot(self.wIN,x)
        self.hF = self.hI + (1/self.τ)*(-self.hI + f(self.u))
        yP = np.dot(self.wOUT, self.hF)
        e = y - yP

        self.p = np.outer(d_f(self.u),self.hI)/self.τ + (1-1/self.τ)*self.p
        self.q = np.outer(d_f(self.u),self.xI)/self.τ + (1-1/self.τ)*self.q
        #print('self.p:',self.p)
        #print('self.q:',self.q)
        δOUT = η1*np.outer(e,self.hF)
        δHS = η2*np.outer(np.dot(self.b, e),np.ones(self.nHS))*self.p
        δIN = η3*np.outer(np.dot(self.b, e),np.ones(self.nIN))*self.q

        self.ΔOUT = (self.ΔOUT*self.k/(self.k+1)) + (δOUT/(self.k+1))
        self.ΔHS = (self.ΔHS*self.k/(self.k+1)) + (δHS/(self.k+1))
        self.ΔIN =  (self.ΔIN*self.k/(self.k+1)) + (δIN/(self.k+1))

        #print('antes self.wIN:',self.wIN)
        self.wIN = self.wIN + self.ΔIN
        self.wHS = self.wHS + self.ΔHS
        self.wOUT = self.wOUT + self.ΔOUT
        #print('depois self.wIN:',self.wIN)

        #print('antes:',self.hI)
        self.hI = self.hF
        self.ht = self.hF
        self.ht2 = self.hF
        self.xI = x
        #print('depois:',self.hI)

    def predict(self,x):
        #print(self.ht)
        #print('.......')
        u = np.dot(self.wHS, self.ht) + np.dot(self.wIN, x)
        h = self.ht + (-self.ht + f(u))/self.τ
        y = np.dot(self.wOUT, h)
        self.ht = h
        return y
    
    def restore(self):
        self.ht = self.hF
        self.ht2 = self.hF
    
    def xavier_uniform(self,shape):
        np.random.seed(42)
        n_in, n_out = shape
        limit = np.sqrt(6 / (n_in + n_out))
        return np.random.uniform(-limit, limit, size=shape)


In [98]:
path = r"C:\Artigo_J3C_V2\VMD10\HI\Bearing1_1.csv"
df = pd.read_csv(path)
sig = df['HIX'].values

nIN, nHS,nOUT = 3,3,3
n = 3

data = np.array([sig[i:i+nIN] for i in range(len(sig)-nIN+1)])
X = data[:-1]
Y = data[1:]
X2 = np.array([X[i:i+n] for i in range(len(X)-n+1)])
Y2 = np.array([Y[i:i+n] for i in range(len(Y)-n+1)])


In [99]:
yP,yR=np.array([]),np.array([])
rnn = RNN(nIN,nHS,nOUT)
for x,y in zip(X,Y):
    y_p = rnn.predict(x)
    yP = np.append(yP,y_p[-1])
    yR = np.append(yR,y[-1])
    rnn.fit(x,y)

In [100]:
yP2,yR2=np.array([]),np.array([])
rnn = RNN(nIN,nHS,nOUT)
for x2,y2 in zip(X2,Y2):
    y_p = rnn.predict(x2[-1])
    yP2 = np.append(yP2,y_p[-1])
    yR2 = np.append(yR2,y2[-1][-1])
    #print(x2[-1],y_p)
    #print(yR)
    for x,y in zip(x2,y2):
        rnn.fit(x,y)